### initlization

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from pyspark.sql.functions import trim, col

### Reading From Bronze Table

In [0]:
df= spark.table("data_engineering_project.bronze.crm_cust_info")

### Silver Data Transformation

**Trimming**

In [0]:
for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))

**Normalization**

In [0]:
df = (
    df
    .withColumn(
        "cst_marital_status",
        F.when(F.upper(F.col("cst_marital_status")) == "S", "Single")
         .when(F.upper(F.col("cst_marital_status")) == "M", "Married")
         .otherwise("n/a")
    )
    .withColumn(
        "cst_gndr",
        F.when(F.upper(F.col("cst_gndr")) == "F", "Female")
         .when(F.upper(F.col("cst_gndr")) == "M", "Male")
         .otherwise("n/a")
    )
)

**Remove Records with Missing Customer ID**

In [0]:
df = df.filter(col("cst_id").isNotNull())

**Renaming Columns**

In [0]:
RENAME_MAP = {
    "cst_id": "customer_id",
    "cst_key": "customer_number",
    "cst_firstname": "first_name",
    "cst_lastname": "last_name",
    "cst_marital_status": "marital_status",
    "cst_gndr": "gender",
    "cst_create_date": "created_date"
}
for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)

### Write into Silver Tbale

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("data_engineering_project.silver.crm_customers")

**check silver Table**

In [0]:

%sql
SELECT * FROM data_engineering_project.silver.crm_customers LIMIT 10